In [1]:
print('hello')

hello


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ['GROQ_API_KEY']
hf_token_api = os.environ['HF_TOKEN']
pinecone_api_key = os.environ['PINECONE_API_KEY']

In [2]:
from langchain_groq import ChatGroq
Model = ChatGroq(
    model="llama-3.3-70b-versatile"
)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template('Tell me a curious fact about {soccer_player}')
output_parser = StrOutputParser()

chain = prompt | Model | output_parser

In [4]:
chain.invoke({'soccer_player': 'Ronaldo'})

"A curious fact about Cristiano Ronaldo is that he has a museum dedicated to himself, called the 'CR7 Museum', which is located in his hometown of Funchal, Madeira, Portugal. The museum showcases his achievements, awards, and memorabilia from throughout his career, including his numerous championship trophies and individual awards, such as his Ballon d'Or awards."

### Use of .bind() to add arguments to a Runnable in LCEL Chain

In [ ]:
## stop the execution when the specific stop word comming
chain = prompt | Model.bind(stop = ['Ronaldo']) | output_parser

In [ ]:
chain.invoke({'soccer_player': 'Ronaldo'}) 

'One curious fact about Cristiano '

### Combinig LCEL Chain

In [7]:
from langchain_groq import ChatGroq
Model = ChatGroq(
    model="llama-3.3-70b-versatile"
)

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template('Tell me a sentence about {politician}')
output_parser = StrOutputParser()

chain = prompt | Model | output_parser

In [9]:
chain.invoke('Chamberlain')

'Neville Chamberlain, the former Prime Minister of the United Kingdom, is often remembered for his policy of appeasement towards Nazi Germany, which ultimately failed to prevent the outbreak of World War II.'

# Combined Chain

In [10]:
historian_prompt = ChatPromptTemplate.from_template('Was {politician} positive for Humanity')

composed_chain = {'politician' :  chain} | historian_prompt | Model | output_parser

In [11]:
composed_chain.invoke({'politician' : 'Lincoln'})

"Yes, Abraham Lincoln, the 16th President of the United States, is indeed widely regarded as one of the most influential leaders in American history, and his impact on humanity has been overwhelmingly positive. Here are some reasons why:\n\n1. **Abolition of Slavery**: Lincoln's strong stance against slavery and his commitment to ending the institution are well-documented. He issued the Emancipation Proclamation in 1863, which declared freedom for all slaves in the Confederate states. This move paved the way for the eventual abolition of slavery with the passage of the 13th Amendment to the Constitution in 1865.\n2. **Unification of the Country**: Lincoln's leadership during the American Civil War (1861-1865) helped to preserve the Union and prevent the country from being torn apart by secession. His commitment to unity and his vision for a stronger, more unified America have had a lasting impact on the country's development.\n3. **Advancement of Civil Rights**: Lincoln's commitment to

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from operator import itemgetter

prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}")

Model = ChatGroq(
    model="llama-3.3-70b-versatile"
)

chain1 = prompt1 | Model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | Model
    | StrOutputParser()
)

chain2.invoke({"politician": "Mitterrand", "language": "French"})


'Le pays de François Mitterrand, la France, est situé sur le continent européen.'

### LCEL Chain at Work in a Typical RAG app

In [14]:
from langchain.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough ,RunnableParallel
from langchain import hub
from langsmith import Client
from bs4 import SoupStrainer
import bs4

# Définition du loader pour extraire uniquement certaines parties du HTML
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only = bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

# Chargement du contenu web
docs = loader.load()

# Découpage du texte en chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_embeddings = HuggingFaceEmbeddings(model_name=model_name)

vector_store = FAISS.from_documents(documents=splits, embedding=model_embeddings)

retriever = vector_store.as_retriever()

template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:

"""
prompt = ChatPromptTemplate.from_template(template=template)
# prompt = hub.pull('rlm/rag-prompt')
# prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context" : retriever | format_docs, "question" : RunnablePassthrough()}
    | prompt
    | Model
    | output_parser
)

In [15]:
rag_chain.invoke('What is Task Decomposition ?')

'Task decomposition is the process of breaking down a complex task into smaller, simpler steps. This can be done through various methods, including using large language models (LLMs) with simple prompting, task-specific instructions, or human inputs. The goal of task decomposition is to transform big tasks into multiple manageable tasks, making it easier to plan and execute the task.'